# IFSC Analysis
## Import Libraries & Tools

In [1]:
import pandas as pd
import numpy as np
import glob
import os

## Setting Up Directory Structure

In [2]:
DATA_DIR = os.path.join(
    os.path.dirname(os.path.realpath("__file__")), "data"
)

BOULDER_MEN_DIR   = os.path.join(DATA_DIR, "Boulder/Men")
BOULDER_WOMEN_DIR = os.path.join(DATA_DIR, "Boulder/Women")

## Helper Functions

In [3]:
def clean_boulder(df):
    # Gets tops and zones for qualifications
    df[['Q_Top', 'Q_Zone']] = df['Qualification'].str.split('T', expand=True)
    df['Q_Zone'] = df['Q_Zone'].str.split('Z', expand=True)[0]
    df[['Qualification', 'Q_Top_Att', 'Q_Zone_Att']] = df['Qualification'].str.split(expand=True)

    # Gets tops and zones for semi-finals
    df[['S_Top', 'S_Zone']] = df['Semi-Final'].str.split('T', expand=True)
    df['S_Zone'] = df['S_Zone'].str.split('Z', expand=True)[0]
    df[['Semi-Final', 'S_Top_Att', 'S_Zone_Att']] = df['Semi-Final'].str.split(expand=True)

    # Gets tops and zones for finals
    df[['F_Top', 'F_Zone']] = df['Final'].str.split('T', expand=True)
    df['F_Zone'] = df['F_Zone'].str.split('Z', expand=True)[0]
    df[['Final', 'F_Top_Att', 'F_Zone_Att']] = df['Final'].str.split(expand=True)

    new_cols = ['Q_Top', 'Q_Zone', 'Q_Top_Att', 'Q_Zone_Att',
                'S_Top', 'S_Zone', 'S_Top_Att', 'S_Zone_Att',
                'F_Top', 'F_Zone', 'F_Top_Att', 'F_Zone_Att']

    # Convert all new columns to int
    for col in new_cols:
        df[col] = df[col].astype('float', errors = 'ignore')

    return df

# Function to correct the old scoring syntax
def convert_score(score):
    if score:
        tmp = str(score)
        if 'b' in tmp.lower():
            tops, top_att = tmp.lower().split()[0].split('t')
            zones, zone_att = tmp.lower().split()[1].split('b')

            new_score = tops + 'T' + zones + 'Z ' + top_att + ' ' + zone_att
            return new_score
        return score
    return score

# Function to display a single climber's results
def check_climber(df, name):
    name = name.upper()
    individual = df.groupby(df['Name']).get_group(name)
    return individual

# Function to show the count of a climber's round appearances
def count_round_appearances(df, name):
    name = name.upper()
    # Gets number of time climber appeared for each round
    q = df.groupby(df['Name']).get_group(name).count()['Qualification']
    s = df.groupby(df['Name']).get_group(name).count()['Semi-Final']
    f = df.groupby(df['Name']).get_group(name).count()['Final']
    
    print(f'{name} appearances:')
    print(f'{"Qualification round:" :>20} {q :>3}')
    print(f'{"Semi-Final round:" :>20} {s :>3}')
    print(f'{"Final round:" :>20} {f :>3}')
    
def get_bouldering_results(directory):
    # Grabs all .csv files in given directory and combine into dataframe
    all_files = glob.glob(os.path.join(directory, "*.csv"))
    df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
    
    # Clean up some column/datatype issues
    df['Rank'] = df['Rank'].fillna(-1).astype('int') # Fixes DNS rank
    df['Qualification'] = df['Qualification'].apply(convert_score) # Fixes old scoring syntax
    df['Semi-Final'] = df['Semi-Final'].apply(convert_score)
    df['Final'] = df['Final'].apply(convert_score)
    return df

## Import Scraped Data

In [4]:
boulder_men_all_results = get_bouldering_results(BOULDER_MEN_DIR)
boulder_men_all_results

,Event,ID,Rank,Name,Country,Qualification,Semi-Final,Final
0,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,1204,1,KILIAN FISCHHUBER,AUT,6T6Z 11 9,4T4Z 11 8,4T4Z 8 5
1,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,3577,2,JORG VERHOEVEN,NED,5T6Z 9 10,3T4Z 5 4,4T4Z 9 5
2,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,4783,3,AKITO MATSUSHIMA,JPN,3T6Z 4 15,3T3Z 10 3,2T2Z 7 4
3,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,1213,4,DAVID LAMA,AUT,4T6Z 7 9,3T4Z 7 8,2T4Z 8 10
4,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,849,5,GABRIELE MORONI,ITA,4T5Z 11 11,3T4Z 5 5,1T3Z 3 4
...,...,...,...,...,...,...,...,...
7786,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",3665,48,LUKE GOH WEN BIN,SGP,0T1Z 0 2,NaN,NaN
7787,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",11859,48,DOHYEON KIM,KOR,0T1Z 0 2,NaN,NaN
7788,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",2120,51,SUNGHOON PARK,KOR,0T1Z 0 5,NaN,NaN
7789,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",13548,52,MUHAMMAD FERZA FERNADA ABDI,INA,0T0Z 0 0,NaN,NaN


As we can see from the above, there are almost 7800 rows of data across *all* of the World Cup event we scraped earlier. If we want to look at the results of just a single climber, or perhaps get the number of times that climber appeared in each round of events, we can run the following functions.

In [5]:
individual = check_climber(boulder_men_all_results, 'minyoung lee')
individual.head(10)

,Event,ID,Rank,Name,Country,Qualification,Semi-Final,Final
6078,IFSC Climbing World Championships - Innsbruck ...,2057,92,MINYOUNG LEE,KOR,0T2Z 0 9,NaN,NaN
6798,"IFSC Climbing Worldcup (B,S) - Wujiang (CHN) 2019",2057,59,MINYOUNG LEE,KOR,2T4Z 3 6,NaN,NaN
7160,"IFSC - Climbing World Championships (B,L,S) - ...",2057,12,MINYOUNG LEE,KOR,3T5Z 11 16,1T3Z 4 9,NaN
7421,IFSC - Climbing World Cup (B) - Brixen (ITA) 2022,2057,59,MINYOUNG LEE,KOR,0T3Z 0 16,NaN,NaN
7721,"IFSC - Climbing World Cup (B,S) - Salt Lake Ci...",2057,53,MINYOUNG LEE,KOR,1T3Z 1 3,NaN,NaN
7784,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",2057,45,MINYOUNG LEE,KOR,0T1Z 0 1,NaN,NaN


In [6]:
count_round_appearances(boulder_men_all_results, 'minyoung lee')

MINYOUNG LEE appearances:
Qualification round:   6
   Semi-Final round:   1
        Final round:   0


# Climber Statistics

Now we can look at the entirety of the results for men's bouldering (the exact same process can be followed for women's bouldering). The following function first checks to see if we have a .csv file that contains all of the data already. If we do, it loads it. If not, then it creates a new one - though it does take some time because it does require a complete scrape of the IFSC site to try and get each climber's height.

In [7]:
def get_bouldering_stats(df, category='men'):
    # Checks if we've already created this file
    if category.lower() == 'men':
        csv = 'boulder_stats_men.csv'
    else:
        csv = 'boulder_stats_women.csv'
    
    file = DATA_DIR + '\\' + csv
    print(f'Checking to see if {csv} exists...')
        
    if os.path.exists(file):
        print(f'File found! Loading {csv}...')
        return pd.read_csv(file, index_col = 0)
    
    print('File does not exist - creating it now...')
    
    # Create a new dataframe with athlete's ID, name, and country
    clean_df = df.drop_duplicates(subset=['Name', 'ID'])[['ID', 'Name', 'Country']]

    # Counts number of appearances per round per climber
    qual_all = df.groupby(['ID', 'Name'])['Qualification'].count()
    semi_app = df.groupby(['ID', 'Name'])['Semi-Final'].count()
    final_app = df.groupby(['ID', 'Name'])['Final'].count()

    # Merges the above dataframes
    rounds = pd.merge(pd.merge(qual_all, semi_app , left_on=['ID', 'Name'], right_index=True), final_app, left_on=['ID', 'Name'], right_index=True)
    clean_df = pd.merge(clean_df, rounds, left_on=['ID', 'Name'], right_index=True)

    # Calculates percentages of appearances in each round
    clean_df['Q_Pct'] = round((clean_df['Qualification'] / clean_df['Qualification']) * 100, 2)
    clean_df['S_Pct'] = round((clean_df['Semi-Final'] / clean_df['Qualification']) * 100, 2)
    clean_df['F_Pct'] = round((clean_df['Final'] / clean_df['Qualification']) * 100, 2)
    clean_df = clean_df.sort_values('Final', ascending=False).reset_index(drop=True)

    # Convert 'ID' to string so we can scrape site for heights (~33 minutes)
    print(f'Gathering {category}\'s height - please be patient...')
    clean_df['ID'] = clean_df['ID'].apply(str)
    scraper = IFSCScraper()
    clean_df['Height'] = clean_df['ID'].apply(lambda x: scraper.get_athlete_height(x))
    scraper.end_session()

    # Saves data so we don't have to scrape later    
    clean_df.to_csv(file)
    print('...Done!')
    
    return clean_df

boulder_men_stats = get_bouldering_stats(boulder_men_all_results, 'men')

# Number of climbers that have a height listed on the IFSC site
have_height_men = (~boulder_men_stats['Height'].str.contains('-')).sum()
no_height_men = (boulder_men_stats['Height'].str.contains('-')).sum()
print(f'Only {have_height_men} male climbers ({(have_height_men/no_height_men)*100:.2f}%) have their height listed!')

boulder_men_stats

Checking to see if boulder_stats_men.csv exists...
File found! Loading boulder_stats_men.csv...
Only 153 male climbers (13.32%) have their height listed!


,ID,Name,Country,Qualification,Semi-Final,Final,Q_Pct,S_Pct,F_Pct,Height
0,1204,KILIAN FISCHHUBER,AUT,56,51,44,100.0,91.07,78.57,-
1,53,DMITRII SHARAFUTDINOV,RUS,65,53,40,100.0,81.54,61.54,-
2,60,RUSTAM GELMANOV,RUS,69,56,38,100.0,81.16,55.07,-
3,2272,KOKORO FUJII,JPN,54,48,27,100.0,88.89,50.00,176
4,79,ALEKSEY RUBTSOV,RUS,57,50,26,100.0,87.72,45.61,178
...,...,...,...,...,...,...,...,...,...,...
1297,6063,DAVID WETMORE,USA,2,0,0,100.0,0.00,0.00,-
1298,1736,AUSTIN GEIMAN,USA,15,1,0,100.0,6.67,0.00,-
1299,6081,WOLTER WESTDIJK,NED,1,0,0,100.0,0.00,0.00,-
1300,6080,MELLE VROOM,NED,1,0,0,100.0,0.00,0.00,-


As you can see from the above, from our original ~7800 rows, we have condensed it down to 1302 unique male climbers. Along with their respective countries of origin, you can see the number of times each climber appeared in the qualification, semi-final, and final rounds. Additionally, you can see a percentage of times the climber reached that specific round. We're not done yet however, as there is still some important information we can figure out.

## Podium Counts?
How many times did each climber reach the podium? How many Bronze, Silver, or Gold medals did they get? Now that we have all of the data in a single dataframe, we can find out! To do this, we can use the [.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) method, as well as a clever use of the [.size](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.size.html?highlight=size#pandas.core.groupby.DataFrameGroupBy.size) and [.unstack](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html?highlight=unstack#pandas.DataFrame.unstack) methods.

In [8]:
def calculate_podiums(all_df, stats_df, country = False):
    # Get a dataframe of all climbers who had a rank of 1, 2, or 3 in any event
    podiums = all_df[all_df['Rank'] <= 3]

    # Get another new dataframe with the count of occurences each climber got each rank
    if country:
        medals = podiums.groupby(['Rank', 'Country']).size().unstack(0)
    else:
        medals = podiums.groupby(['ID', 'Name', 'Rank']).size().unstack()
    medals = medals.rename(columns={-1: 'Podium', 1: 'Gold', 2: 'Silver', 3: 'Bronze'})
    medals['Podium'] = medals['Gold'] + medals['Silver'] + medals['Bronze']

    # Merge with our stats dataframe and add new % columns
    if country:
        stats_df           = stats_df.merge(medals, how='left', left_on=['Country'], right_on=['Country']).fillna(0)
    else:
        stats_df           = stats_df.merge(medals, how='left', left_on=['ID', 'Name'], right_on=['ID', 'Name']).fillna(0)
    stats_df['P_Pct']      = round((stats_df['Podium'] / stats_df['Qualification']) * 100, 2)
    stats_df['Gold_Pct']   = round((stats_df['Gold'] / stats_df['Qualification']) * 100, 2)
    stats_df['Silver_Pct'] = round((stats_df['Silver'] / stats_df['Qualification']) * 100, 2)
    stats_df['Bronze_Pct'] = round((stats_df['Bronze'] / stats_df['Qualification']) * 100, 2)
    
    # Convert these columns to 'int'
    int_col = ['Podium', 'Bronze', 'Silver', 'Gold']
    stats_df[int_col] = stats_df[int_col].astype(int)

    # Reordering the columns into a more logical format
    if country:
        cols = ['ID', 'Name', 'Country', 'Height', 'Qualification', 'Semi-Final',
                'Final', 'Podium', 'Bronze', 'Silver', 'Gold', 'Q_Pct', 'S_Pct',
                'F_Pct', 'P_Pct', 'Bronze_Pct', 'Silver_Pct', 'Gold_Pct']
    else:
        cols = ['Country', 'Qualification', 'Semi-Final', 'Final', 'Podium',
                'Bronze', 'Silver', 'Gold', 'Q_Pct', 'S_Pct', 'F_Pct', 'P_Pct',
                'Bronze_Pct', 'Silver_Pct', 'Gold_Pct']

    stats_df = stats_df[cols]
    return stats_df

boulder_men_stats = calculate_podiums(boulder_men_all_results, boulder_men_stats)
boulder_men_stats

,Country,Qualification,Semi-Final,Final,Podium,Bronze,Silver,Gold,Q_Pct,S_Pct,F_Pct,P_Pct,Bronze_Pct,Silver_Pct,Gold_Pct
0,AUT,56,51,44,35,10,8,17,100.0,91.07,78.57,62.50,17.86,14.29,30.36
1,RUS,65,53,40,26,3,9,14,100.0,81.54,61.54,40.00,4.62,13.85,21.54
2,RUS,69,56,38,19,7,6,6,100.0,81.16,55.07,27.54,10.14,8.70,8.70
3,JPN,54,48,27,0,3,0,6,100.0,88.89,50.00,0.00,5.56,0.00,11.11
4,RUS,57,50,26,14,10,1,3,100.0,87.72,45.61,24.56,17.54,1.75,5.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,USA,2,0,0,0,0,0,0,100.0,0.00,0.00,0.00,0.00,0.00,0.00
1298,USA,15,1,0,0,0,0,0,100.0,6.67,0.00,0.00,0.00,0.00,0.00
1299,NED,1,0,0,0,0,0,0,100.0,0.00,0.00,0.00,0.00,0.00,0.00
1300,NED,1,0,0,0,0,0,0,100.0,0.00,0.00,0.00,0.00,0.00,0.00


Now we have a more robust dataframe with more relevant information. Additionally, we can use this dataframe with our previous function to check out a single climber and see all their stats. A reminder that these stats only represent the IFSC World Cup events.

In [9]:
individual2 = check_climber(boulder_men_stats, 'jongwon chon')
individual2

KeyError: 'Name'

# Country Statistics

In [ ]:
def get_bouldering_country_stats(df):
    clean_df = pd.DataFrame()
    clean_df['Country'] = df['Country'].unique()

    # Counts number of appearances per round per country
    qual_all = df.groupby(['Country'])['Qualification'].count()
    semi_app = df.groupby(['Country'])['Semi-Final'].count()
    final_app = df.groupby(['Country'])['Final'].count()

    # Merges the above with our previous dataframe
    clean_df = pd.merge(pd.merge(qual_all, semi_app , left_on='Country', right_index=True),
                  final_app, left_on='Country', right_index=True)

    # Calculates percentages of appearances vs total events
    clean_df = clean_df.sort_values('Final', ascending=False).reset_index()
    clean_df['Q_Pct'] = round((clean_df['Qualification'] / clean_df['Qualification'].sum()) * 100, 2)
    clean_df['S_Pct'] = round((clean_df['Semi-Final'] / clean_df['Semi-Final'].sum()) * 100, 2)
    clean_df['F_Pct'] = round((clean_df['Final'] / clean_df['Final'].sum()) * 100, 2)
    return clean_df

boulder_men_country_stats = get_bouldering_country_stats(boulder_men_all_results)
boulder_men_country_stats

We can then re-configure our previous function that calculated podium results for individual climbers for countries.

In [ ]:
calculate_podiums(boulder_men_all_results, boulder_men_country_stats, True)